In [4]:
%load_ext autoreload
%autoreload 2

import os
import yaml
import time
import torch
import random
import numpy as np

from torch.utils import data

from tensorboardX import SummaryWriter

import data_loader
import models
from optimizer import get_optimizer
from loss.cross_entropy_2d import CrossEntropy2d

SEED = 1337

def train(config, writer=None):
    # setup seeds
    torch.manual_seed(config["seed"])
    torch.cuda.manual_seed(config["seed"])
    np.random.seed(config["seed"])

    # setup device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # setup augmentations
    ## ah todo
    
    # setup dataloader
    mit_data = data_loader.get_loader("mit_sceneparsing_benchmark")
    train_data_raw = mit_data("training")
    train_data = data.DataLoader(train_data_raw, batch_size=config["batch_sz"],\
                                 num_workers=1, shuffle=True)
    
    val = data.DataLoader(mit_data("validation"), batch_size=config["batch_sz"],\
                                 num_workers=1)

    # setup metrics
    ## ah todo
    
    # setup model
    model = models.get_model("fcn32s", train_data_raw.n_classes).to(device)
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
    
    # setup optimizer
    optimizer = get_optimizer("adam")(model.parameters())
    
    print(f"Optimizer: {optimizer}")
    
    # setup learning rate scheduler (optim.lr_scheduler)
    ## ah todo
    loss_fn = CrossEntropy2d()
        
    i = 0
    while i < config["epochs"]:
        i += 1
        for (images, labels) in train_data:
            print("f{images.size()} ... {labels.size()}")
            start_ts = time.time()
            model.train()
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            out = model(images)
            
            loss = loss_fn(out, labels)
            
            loss.backward()
            optimizer.step()
            
            print(f"Batch Loss : {loss} ... time : {time.time() - start_ts}")
            
            
config = {"batch_sz": 256, "epochs": 2, "seed": 42}

train(config)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 512 training images
Found 512 validation images
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f7caf6137a0>
Traceback (most recent call last):
  File "/home/ammar/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 962, in __del__
    self._shutdown_workers()
  File "/home/ammar/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 942, in _shutdown_workers
    w.join()
  File "/home/ammar/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


ValueError: not enough values to unpack (expected 2, got 1)